In [1]:
import os

In [ ]:
from collections import defaultdict 

targetDir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder3'

def obtainSizeRange(targetDir, desiredInputNumber):
    lstDir = os.listdir(targetDir)    
    sizeDict = defaultdict(int)
    for i in lstDir:
        t_number = int(i.split("t-")[1].split("_")[0])
        lowerT = 200 - (200/desiredInputNumber) + 1
        if lowerT <= t_number <= 200:
            size = float(i[-17:-5])
            sizeRange = int(size*2)/2
            sizeDict[sizeRange] += 1

    print(sizeDict)
    print(dict(sorted(sizeDict.items())))
    print(dict(sorted(sizeDict.items(), key=lambda item: item[1])))

obtainSizeRange(targetDir, 4)